In [1]:
import pandas as pd
import numpy as np
from pymatgen.core import Composition
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.model_selection import train_test_split
import math
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
config = {
    "mathtext.fontset":'stix',
    "font.family":'serif',
    "font.serif": ['Arial'],
    "font.size": 24,
    'axes.unicode_minus': False 
}
plt.rcParams.update(config)
plt.rcParams['axes.unicode_minus'] = False  
large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (8, 6),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.rcParams['figure.dpi'] = 400
plt.rcParams['axes.grid'] = False

In [2]:
def get_combined_features(data: pd.DataFrame):
    columns = data.columns
    A_features = []
    B_features = []
    O_features = []
    for column in columns:
        if str(column).__contains__('A'):
            A_features.append(column)
        elif str(column).__contains__('B'):
            B_features.append(column)
        elif str(column).__contains__('O'):
            O_features.append(column)

    for a_feature, b_feature, o_feature in zip(A_features, B_features, O_features):
        feature_name = str(a_feature).split('_')[1]
        feature_name = feature_name
        data['A-B_'+feature_name] = np.abs(data[a_feature] - data[b_feature])
        data['(A+B)-O_'+feature_name] = np.abs((data[a_feature] + data[b_feature])- data[o_feature])


    data = data.drop(columns=O_features)
    return data

In [ ]:
small_data = pd.read_csv('/data/single_perovskite_oxide.csv')
small_data = small_data.drop(columns = 'Unnamed: 0').reset_index(drop=True)
small_data = small_data.drop(columns = 'composition').reset_index(drop=True)
small_data['composition'] = small_data['formula'].map(Composition)
y_eg_abo3 = small_data['target']

In [ ]:
pysic_features_ABO3 = pd.read_csv('/data/pysic_features_ABO3.csv').drop(columns = 'Unnamed: 0')
pysic_labels = pysic_features_ABO3.columns.drop('formula')
pysic_labels

Index(['A_Density', 'B_Density', 'O_Density', 'A_dipole Polarizability',
       'B_dipole Polarizability', 'O_dipole Polarizability',
       'A_covalent Radius', 'B_covalent Radius', 'O_covalent Radius',
       'A_atomic Radius', 'B_atomic Radius', 'O_atomic Radius',
       'A_FirstIonization', 'B_FirstIonization', 'O_FirstIonization',
       'A_number of Valence Electrons', 'B_number of Valence Electrons',
       'O_number of Valence Electrons', 'A_number', 'B_number', 'O_number',
       'A_Period', 'B_Period', 'O_Period', 'A_Electronegativity',
       'B_Electronegativity', 'O_Electronegativity',
       'A_number of s+p Electrons', 'B_number of s+p Electrons',
       'O_number of s+p Electrons', 'A_number of d Electrons',
       'B_number of d Electrons', 'O_number of d Electrons', 'A_Mulliken EN',
       'B_Mulliken EN', 'O_Mulliken EN'],
      dtype='object')

In [5]:
pysic_features_ABO3 = pysic_features_ABO3[pysic_labels]
pysic_features_ABO3 = get_combined_features(pysic_features_ABO3)
pysic_features_ABO3

,A_Density,B_Density,A_dipole Polarizability,B_dipole Polarizability,A_covalent Radius,B_covalent Radius,A_atomic Radius,B_atomic Radius,A_FirstIonization,B_FirstIonization,...,A-B_Period,(A+B)-O_Period,A-B_Electronegativity,(A+B)-O_Electronegativity,A-B_number of s+p Electrons,(A+B)-O_number of s+p Electrons,A-B_number of d Electrons,(A+B)-O_number of d Electrons,A-B_Mulliken EN,(A+B)-O_Mulliken EN
0,7.520,7.15,192.0,83.00,172,122,1.85,1.40,544.5,652.9,...,2,8,0.49,0.61,1,3,5,5,0.62,0.72
1,2.990,8.86,97.0,55.00,148,111,1.60,1.35,633.1,760.4,...,0,6,0.52,0.20,0,2,6,8,0.96,0.10
2,7.900,7.15,158.0,83.00,169,122,1.80,1.40,593.4,652.9,...,2,8,0.46,0.58,1,3,4,6,0.62,0.72
3,4.470,7.15,162.0,83.00,163,122,1.80,1.40,600.0,652.9,...,1,7,0.44,0.56,1,3,4,6,0.53,0.63
4,6.150,7.87,215.0,62.00,180,116,1.95,1.40,538.1,762.5,...,2,8,0.73,0.51,0,2,5,7,0.96,0.38
5,7.520,8.86,192.0,55.00,172,111,1.85,1.35,544.5,760.4,...,2,8,0.71,0.39,0,2,7,7,1.20,0.14
6,4.470,7.87,162.0,62.00,163,116,1.80,1.40,600.0,762.5,...,1,7,0.61,0.39,0,2,5,7,0.87,0.29
7,6.150,7.15,215.0,83.00,180,122,1.95,1.40,538.1,652.9,...,2,8,0.56,0.68,1,3,4,6,0.62,0.72
8,6.770,7.15,216.0,83.00,176,122,1.85,1.40,527.0,652.9,...,2,8,0.53,0.65,1,3,5,5,0.62,0.72
9,7.010,7.15,208.0,83.00,174,122,1.85,1.40,533.1,652.9,...,2,8,0.52,0.64,1,3,5,5,0.62,0.72


In [ ]:
re_data = pd.read_csv('/data/all_features_df.csv')

In [7]:
re_data

,formula,A1_x,A1_ratio_x,nA1_x,A2_x,A2_ratio_x,nA2_x,B1_x,B1_ratio_x,B2_x,...,B_atomic_ie_1,B_atomic_ie_2,B_atomic_ndunf,B_atomic_ndval,B_atomic_npunf,B_atomic_npval,B_atomic_nunf,B_atomic_nval,B_atomic_phi,B_atomic_pol
0,Sm0.2Sc0.8Cr1O3,Sm,0.2,3,Sc,0.8,3,Cr,1.0,Cr,...,6.766510,16.485700,5.0,5.0,0.0,0.0,6.0,6.0,4.650,11.600
1,Sm0.2Sc0.8Fe1O3,Sm,0.2,3,Sc,0.8,3,Fe,1.0,Fe,...,7.902400,16.187700,4.0,6.0,0.0,0.0,4.0,8.0,4.930,8.400
2,Sm0.2Zr1Sc0.8O3,Sm,0.2,3,Sc,0.8,3,Zr,1.0,Zr,...,6.633900,13.100000,8.0,2.0,0.0,0.0,8.0,4.0,3.450,17.900
3,Sm0.2Sc0.8Ge1O3,Sm,0.2,3,Sc,0.8,3,Ge,1.0,Ge,...,7.899430,15.934610,0.0,10.0,4.0,2.0,4.0,14.0,4.550,5.840
4,Sm0.2Hf1Sc0.8O3,Sm,0.2,3,Sc,0.8,3,Hf,1.0,Hf,...,6.825070,15.000000,8.0,2.0,0.0,0.0,8.0,18.0,3.600,16.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35320,Sr1Ce0.7Zr0.3O3,Sr,1.0,2,Sr,0.0,2,Ce,0.7,Zr,...,5.867260,11.525000,8.7,1.3,0.0,0.0,17.8,4.0,3.261,26.090
35321,Sr1Ce0.7Be0.3O3,Sr,1.0,2,Sr,0.0,2,Ce,0.7,Be,...,6.673900,13.058342,6.3,0.7,0.0,0.0,15.4,3.4,3.741,22.400
35322,Sr1Ce0.7Ge0.3O3,Sr,1.0,2,Sr,0.0,2,Ce,0.7,Ge,...,6.246919,12.375383,6.3,3.7,1.2,0.6,16.6,7.0,3.591,22.472
35323,Sr1Ce0.7Hf0.3O3,Sr,1.0,2,Sr,0.0,2,Ce,0.7,Hf,...,5.924611,12.095000,8.7,1.3,0.0,0.0,17.8,8.2,3.306,25.580


In [8]:
def get_combined_features(data: pd.DataFrame):
    columns = data.columns
    A_features = []
    B_features = []
    O_features = []
    for column in columns:
        if str(column).__contains__('A'):
            A_features.append(column)
        elif str(column).__contains__('B'):
            B_features.append(column)
        elif str(column).__contains__('O'):
            O_features.append(column)

    for a_feature, b_feature, o_feature in zip(A_features, B_features, O_features):
        feature_name = str(a_feature).split('_')[1]
        feature_name = feature_name
        data['A-B_'+feature_name] = np.abs(data[a_feature] - data[b_feature])
        data['(A+B)-O_'+feature_name] = np.abs((data[a_feature] + data[b_feature])- data[o_feature])


    data = data.drop(columns=O_features)
    return data

In [9]:
pysic_features = re_data[pysic_labels]
pysic_features =  get_combined_features(pysic_features)
pysic_features[['formula','Eg']] = re_data[['formula','Eg']]
pysic_features

,A_Density,B_Density,A_dipole Polarizability,B_dipole Polarizability,A_covalent Radius,B_covalent Radius,A_atomic Radius,B_atomic Radius,A_FirstIonization,B_FirstIonization,...,A-B_Electronegativity,(A+B)-O_Electronegativity,A-B_number of s+p Electrons,(A+B)-O_number of s+p Electrons,A-B_number of d Electrons,(A+B)-O_number of d Electrons,A-B_Mulliken EN,(A+B)-O_Mulliken EN,formula,Eg
0,3.896,7.150,116.0,83.000,152.8,122.0,1.65,1.400,615.38,652.90,...,0.338,0.458,1.0,3.0,4.2,5.8,0.428,0.528,Sm0.2Sc0.8Cr1O3,2.146025
1,3.896,7.870,116.0,62.000,152.8,116.0,1.65,1.400,615.38,762.50,...,0.508,0.288,0.0,2.0,5.2,6.8,0.768,0.188,Sm0.2Sc0.8Fe1O3,1.374682
2,3.896,6.510,116.0,112.000,152.8,154.0,1.65,1.550,615.38,640.10,...,0.008,0.788,0.0,2.0,1.2,2.8,0.348,0.608,Sm0.2Zr1Sc0.8O3,4.177025
3,3.896,5.320,116.0,40.000,152.8,121.0,1.65,1.250,615.38,762.00,...,0.688,0.108,2.0,0.0,0.8,0.8,1.308,0.352,Sm0.2Sc0.8Ge1O3,3.941249
4,3.896,13.300,116.0,103.000,152.8,152.0,1.65,1.550,615.38,658.50,...,0.022,0.818,0.0,2.0,1.2,2.8,0.508,0.448,Sm0.2Hf1Sc0.8O3,4.529786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35320,2.640,6.692,197.2,177.100,185.0,160.3,2.00,1.760,549.50,566.11,...,0.233,1.307,0.0,2.0,1.3,1.3,1.262,2.278,Sr1Ce0.7Zr0.3O3,3.843865
35321,2.640,5.294,197.2,154.822,185.0,144.7,2.00,1.610,549.50,643.93,...,0.305,1.235,0.0,2.0,0.7,0.7,1.640,1.900,Sr1Ce0.7Be0.3O3,3.431016
35322,2.640,6.335,197.2,155.500,185.0,150.4,2.00,1.670,549.50,602.68,...,0.437,1.103,0.6,1.4,0.7,0.7,1.550,1.990,Sr1Ce0.7Ge0.3O3,2.085930
35323,2.640,8.729,197.2,174.400,185.0,159.7,2.00,1.760,549.50,571.63,...,0.224,1.316,0.0,2.0,1.3,1.3,1.310,2.230,Sr1Ce0.7Hf0.3O3,3.646532


In [10]:
p_label = pysic_features.columns.drop(['formula', 'Eg'])
p_label

Index(['A_Density', 'B_Density', 'A_dipole Polarizability',
       'B_dipole Polarizability', 'A_covalent Radius', 'B_covalent Radius',
       'A_atomic Radius', 'B_atomic Radius', 'A_FirstIonization',
       'B_FirstIonization', 'A_number of Valence Electrons',
       'B_number of Valence Electrons', 'A_number', 'B_number', 'A_Period',
       'B_Period', 'A_Electronegativity', 'B_Electronegativity',
       'A_number of s+p Electrons', 'B_number of s+p Electrons',
       'A_number of d Electrons', 'B_number of d Electrons', 'A_Mulliken EN',
       'B_Mulliken EN', 'A-B_Density', '(A+B)-O_Density',
       'A-B_dipole Polarizability', '(A+B)-O_dipole Polarizability',
       'A-B_covalent Radius', '(A+B)-O_covalent Radius', 'A-B_atomic Radius',
       '(A+B)-O_atomic Radius', 'A-B_FirstIonization',
       '(A+B)-O_FirstIonization', 'A-B_number of Valence Electrons',
       '(A+B)-O_number of Valence Electrons', 'A-B_number', '(A+B)-O_number',
       'A-B_Period', '(A+B)-O_Period', 'A-B_El

In [12]:
from scipy.stats import spearmanr
def remove_collinear_features(x, y,threshold):
    '''
    Objective:
        Remove collinear features in a dataframe with a correlation coefficient
        greater than the threshold. Removing collinear features can help a model
        to generalize and improves the interpretability of the model.
        
    Inputs: 
        threshold: any features with correlations greater than this value are removed
    
    Output: 
        dataframe that contains only the non-highly-collinear features
    '''  
    # Calculate the correlation matrix
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterate through the correlation matrix and compare correlations
    for i in iters:
        for j in range(i):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)
            
            # If correlation exceeds the threshold
            if val >= threshold:
                # Print the correlated features and the correlation value
#                 print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                x1 = x[col.values[0]]
                x2 = x[row.values[0]]
                corr1,p1 = spearmanr(x1, y)
                corr2,p2 = spearmanr(x2, y)
                if np.abs(corr1) < np.abs(corr2):
                    drop_cols.append(col.values[0])
                else:
                    drop_cols.append(row.values[0])
    # Drop one of each pair of correlated columns
    drops = set(drop_cols)
    x = x.drop(columns = drops)
    return x

# Magpie

In [ ]:
magpie_data = pd.read_csv("/data/magpie_synthetic_data.csv").drop(columns = 'Unnamed: 0')
magpie_data

,formula,Eg,composition,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,MagpieData mode Number,MagpieData minimum MendeleevNumber,...,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char
0,Sm0.2Sc0.8Cr1O3,2.146025,Sm0.2 Sc0.8 Cr1 O3,8.0,62.0,54.0,15.44,8.9280,8.0,11.0,...,90.68,94.416,12.0,1.8,2.40,1.16,0.24,True,0.724240,0.147721
1,Sm0.2Sc0.8Fe1O3,1.374682,Sm0.2 Sc0.8 Fe1 O3,8.0,62.0,54.0,15.84,9.4080,8.0,11.0,...,90.68,94.416,12.0,2.0,2.40,1.36,0.24,True,0.724240,0.140666
2,Sm0.2Zr1Sc0.8O3,4.177025,Sm0.2 Zr1 Sc0.8 O3,8.0,62.0,54.0,18.64,12.7680,8.0,11.0,...,83.68,86.016,12.0,2.0,2.40,0.56,0.24,False,0.724240,0.161521
3,Sm0.2Sc0.8Ge1O3,3.941249,Sm0.2 Sc0.8 Ge1 O3,8.0,62.0,54.0,17.04,10.8480,8.0,11.0,...,89.88,93.456,12.0,2.0,2.80,2.16,0.24,False,0.724240,0.133420
4,Sm0.2Hf1Sc0.8O3,4.529786,Sm0.2 Hf1 Sc0.8 O3,8.0,72.0,64.0,25.04,21.7408,8.0,11.0,...,83.68,86.016,12.0,2.0,2.40,0.56,3.04,False,0.724240,0.162762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35320,Sr1Ce0.7Zr0.3O3,3.843865,Sr1 Ce0.7 Zr0.3 O3,8.0,58.0,50.0,22.92,17.9040,8.0,8.0,...,91.00,94.800,12.0,2.0,2.40,0.26,0.14,True,0.787757,0.181550
35321,Sr1Ce0.7Be0.3O3,3.431016,Sr1 Ce0.7 Be0.3 O3,4.0,58.0,54.0,20.76,17.3232,8.0,8.0,...,91.00,94.800,12.0,2.0,2.40,0.14,0.14,False,0.787757,0.179356
35322,Sr1Ce0.7Ge0.3O3,2.085930,Sr1 Ce0.7 Ge0.3 O3,8.0,58.0,50.0,22.44,17.3280,8.0,8.0,...,92.86,97.032,12.0,2.0,2.52,0.74,0.14,True,0.787757,0.175717
35323,Sr1Ce0.7Hf0.3O3,3.646532,Sr1 Ce0.7 Hf0.3 O3,8.0,72.0,64.0,24.84,20.2080,8.0,8.0,...,91.00,94.800,12.0,2.0,2.40,0.26,0.98,True,0.787757,0.181833


In [14]:
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers import composition as cf
from pymatgen.core import Composition
feature_calculators = MultipleFeaturizer([cf.ElementProperty.from_preset("magpie"),
                                          cf.ValenceOrbital(props=['avg']), cf.IonProperty(fast=True)])
feature_labels = feature_calculators.feature_labels()
small_magpie_features = feature_calculators.featurize_dataframe(small_data,col_id='composition')
small_magpie_features

MultipleFeaturizer:   0%|          | 0/52 [00:00<?, ?it/s]

,formula,target,A,B,composition,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,MagpieData avg_dev Number,...,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char
0,SmCrO3,1.736200,Sm,Cr,"(Sm, Cr, O)",8.0,62.0,54.0,22.0,16.80,...,86.2,89.04,12.0,1.8,2.4,1.0,1.2,True,0.724240,0.154892
1,ScCoO3,0.786000,Sc,Co,"(Sc, Co, O)",8.0,27.0,19.0,14.4,7.68,...,84.8,87.36,12.0,2.0,2.4,1.6,0.0,True,0.660947,0.136622
2,GdCrO3,0.825000,Gd,Cr,"(Gd, Cr, O)",8.0,64.0,56.0,22.4,17.28,...,91.8,95.76,12.0,1.8,2.4,1.2,1.4,True,0.714753,0.153484
3,YCrO3,1.767900,Y,Cr,"(Y, Cr, O)",8.0,39.0,31.0,17.4,11.28,...,91.8,95.76,12.0,1.8,2.4,1.2,0.0,True,0.708321,0.152541
4,LaFeO3,0.848400,La,Fe,"(La, Fe, O)",8.0,57.0,49.0,21.4,16.08,...,91.8,95.76,12.0,2.0,2.4,1.4,0.0,True,0.745613,0.151694
5,SmCoO3,0.803900,Sm,Co,"(Sm, Co, O)",8.0,62.0,54.0,22.6,17.52,...,79.2,80.64,12.0,2.0,2.4,1.4,1.2,True,0.724240,0.146339
6,YFeO3,0.637300,Y,Fe,"(Y, Fe, O)",8.0,39.0,31.0,17.8,11.76,...,91.8,95.76,12.0,2.0,2.4,1.4,0.0,True,0.708321,0.145782
7,LaCrO3,1.903900,La,Cr,"(La, Cr, O)",8.0,57.0,49.0,21.0,15.60,...,91.8,95.76,12.0,1.8,2.4,1.2,0.0,True,0.745613,0.158143
8,PrCrO3,1.847000,Pr,Cr,"(Pr, Cr, O)",8.0,59.0,51.0,21.4,16.08,...,91.8,95.76,12.0,1.8,2.4,1.0,0.6,True,0.736585,0.156756
9,NdCrO3,1.904000,Nd,Cr,"(Nd, Cr, O)",8.0,60.0,52.0,21.6,16.32,...,91.8,95.76,12.0,1.8,2.4,1.0,0.8,True,0.733532,0.156291


# Pysic + Mgapie

In [20]:
data = pysic_features[p_label].copy(deep = True)
data[feature_labels] = magpie_data[feature_labels]
data

,A_Density,B_Density,A_dipole Polarizability,B_dipole Polarizability,A_covalent Radius,B_covalent Radius,A_atomic Radius,B_atomic Radius,A_FirstIonization,B_FirstIonization,...,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char
0,3.896,7.150,116.0,83.000,152.8,122.0,1.65,1.400,615.38,652.90,...,90.68,94.416,12.0,1.8,2.40,1.16,0.24,True,0.724240,0.147721
1,3.896,7.870,116.0,62.000,152.8,116.0,1.65,1.400,615.38,762.50,...,90.68,94.416,12.0,2.0,2.40,1.36,0.24,True,0.724240,0.140666
2,3.896,6.510,116.0,112.000,152.8,154.0,1.65,1.550,615.38,640.10,...,83.68,86.016,12.0,2.0,2.40,0.56,0.24,False,0.724240,0.161521
3,3.896,5.320,116.0,40.000,152.8,121.0,1.65,1.250,615.38,762.00,...,89.88,93.456,12.0,2.0,2.80,2.16,0.24,False,0.724240,0.133420
4,3.896,13.300,116.0,103.000,152.8,152.0,1.65,1.550,615.38,658.50,...,83.68,86.016,12.0,2.0,2.40,0.56,3.04,False,0.724240,0.162762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35320,2.640,6.692,197.2,177.100,185.0,160.3,2.00,1.760,549.50,566.11,...,91.00,94.800,12.0,2.0,2.40,0.26,0.14,True,0.787757,0.181550
35321,2.640,5.294,197.2,154.822,185.0,144.7,2.00,1.610,549.50,643.93,...,91.00,94.800,12.0,2.0,2.40,0.14,0.14,False,0.787757,0.179356
35322,2.640,6.335,197.2,155.500,185.0,150.4,2.00,1.670,549.50,602.68,...,92.86,97.032,12.0,2.0,2.52,0.74,0.14,True,0.787757,0.175717
35323,2.640,8.729,197.2,174.400,185.0,159.7,2.00,1.760,549.50,571.63,...,91.00,94.800,12.0,2.0,2.40,0.26,0.98,True,0.787757,0.181833


# Model

In [21]:
from scipy.stats import spearmanr
def remove_collinear_features(x, y,threshold):
    '''
    Objective:
        Remove collinear features in a dataframe with a correlation coefficient
        greater than the threshold. Removing collinear features can help a model
        to generalize and improves the interpretability of the model.
        
    Inputs: 
        threshold: any features with correlations greater than this value are removed
    
    Output: 
        dataframe that contains only the non-highly-collinear features
    '''  
    # Calculate the correlation matrix
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterate through the correlation matrix and compare correlations
    for i in iters:
        for j in range(i):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)
            
            # If correlation exceeds the threshold
            if val >= threshold:
                # Print the correlated features and the correlation value
#                 print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                x1 = x[col.values[0]]
                x2 = x[row.values[0]]
                corr1,p1 = spearmanr(x1, y)
                corr2,p2 = spearmanr(x2, y)
                if np.abs(corr1) < np.abs(corr2):
                    drop_cols.append(col.values[0])
                else:
                    drop_cols.append(row.values[0])
    # Drop one of each pair of correlated columns
    drops = set(drop_cols)
    x = x.drop(columns = drops)
    return x

In [22]:
X = data.copy(deep = True)
y = re_data['Eg']
X = remove_collinear_features(X,y,0.8)
fl = X.columns
X = data[fl]
X

,A_Density,B_Density,A_covalent Radius,B_atomic Radius,B_FirstIonization,B_number of Valence Electrons,A_number of s+p Electrons,B_number of s+p Electrons,A_number of d Electrons,B_number of d Electrons,...,MagpieData avg_dev GSmagmom,MagpieData mode GSmagmom,MagpieData minimum SpaceGroupNumber,MagpieData maximum SpaceGroupNumber,MagpieData range SpaceGroupNumber,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg f valence electrons,compound possible
0,3.896,7.150,152.8,1.400,652.90,6.0,2.0,1.0,0.8,5.0,...,0.000002,0.0,12.0,229.0,217.0,90.68,94.416,12.0,0.24,True
1,3.896,7.870,152.8,1.400,762.50,8.0,2.0,2.0,0.8,6.0,...,0.675412,0.0,12.0,229.0,217.0,90.68,94.416,12.0,0.24,True
2,3.896,6.510,152.8,1.550,640.10,4.0,2.0,2.0,0.8,2.0,...,0.000002,0.0,12.0,194.0,182.0,83.68,86.016,12.0,0.24,False
3,3.896,5.320,152.8,1.250,762.00,4.0,2.0,4.0,0.8,0.0,...,0.000002,0.0,12.0,225.0,213.0,89.88,93.456,12.0,0.24,False
4,3.896,13.300,152.8,1.550,658.50,4.0,2.0,2.0,0.8,2.0,...,0.000002,0.0,12.0,194.0,182.0,83.68,86.016,12.0,3.04,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35320,2.640,6.692,185.0,1.760,566.11,3.3,2.0,2.0,0.0,1.3,...,0.000000,0.0,12.0,225.0,213.0,91.00,94.800,12.0,0.14,True
35321,2.640,5.294,185.0,1.610,643.93,2.7,2.0,2.0,0.0,0.7,...,0.000000,0.0,12.0,225.0,213.0,91.00,94.800,12.0,0.14,False
35322,2.640,6.335,185.0,1.670,602.68,3.3,2.0,2.6,0.0,0.7,...,0.000000,0.0,12.0,225.0,213.0,92.86,97.032,12.0,0.14,True
35323,2.640,8.729,185.0,1.760,571.63,3.3,2.0,2.0,0.0,1.3,...,0.000000,0.0,12.0,225.0,213.0,91.00,94.800,12.0,0.98,True


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [24]:
# dop_magpie_etr = ExtraTreesRegressor(n_jobs = -1 ,random_state=42)
# dop_magpie_etr.fit(X_train, y_train)
# y_pred = dop_magpie_etr.predict(X_test)
# print('Performance on the test set: MAE = %0.4f' % mean_absolute_error(y_test,y_pred))
# print('Performance on the test set: RMSE = %0.4f' % np.sqrt(mean_squared_error(y_test, y_pred)))
# print('Performance on the test set: R2 SCORE = %0.4f' % r2_score(y_test,y_pred))

Performance on the test set: MAE = 0.3121
Performance on the test set: RMSE = 0.4320
Performance on the test set: R2 SCORE = 0.8783


In [25]:
from sklearn.ensemble import ExtraTreesRegressor
etr = ExtraTreesRegressor(n_jobs=-1, random_state=42)
etr.fit(X_train, y_train)
feature_importance=pd.DataFrame()
feature_importance['features'] = X.columns
feature_importance['feature_importances'] = etr.feature_importances_
threshold = np.sort(feature_importance['feature_importances'].values)
score = []
n_features = []
cols = []
for i in tqdm(threshold):
    selector = SelectFromModel(etr,threshold=i)
    X_embedded = selector.fit_transform(X_train, y_train)
    once = cross_val_score(etr,X_embedded,y_train,cv=5,scoring="r2",n_jobs=-1).mean()#交叉验证
    score.append(once)
    n_features.append(X_embedded.shape[1])
    cols.append(X.columns[selector.get_support()])
result = pd.DataFrame({"col":cols,"score":score,"n_features":n_features})
result

100%|██████████| 99/99 [06:15<00:00,  3.80s/it]


,col,score,n_features
0,"Index(['A_Density', 'B_Density', 'A_covalent R...",0.866623,99
1,"Index(['A_Density', 'B_Density', 'A_covalent R...",0.866623,99
2,"Index(['A_Density', 'B_Density', 'A_covalent R...",0.866623,99
3,"Index(['A_Density', 'B_Density', 'A_covalent R...",0.866623,99
4,"Index(['A_Density', 'B_Density', 'A_covalent R...",0.866623,99
...,...,...,...
94,"Index(['B_number of Valence Electrons', 'B_num...",0.702791,5
95,"Index(['B_number of Valence Electrons', '(A+B)...",0.689843,4
96,"Index(['B_number of Valence Electrons', '(A+B)...",0.581510,3
97,"Index(['B_number of Valence Electrons', 'Magpi...",0.490269,2


In [26]:
features_select = result.sort_values(by='score',ascending=True)
features_select = features_select[features_select['score']>=0.86]

In [27]:
optimize_feature_labels = features_select.iloc[0]['col']
X_train = X.loc[X_train.index][optimize_feature_labels]
X_test = X.loc[X_test.index][optimize_feature_labels]
if 'O' in X_train.columns:
    X_train = X_train.drop(columns='O')
    X_test = X_test.drop(columns='O')
X_train.shape, X_test.shape

((28260, 29), (7065, 29))

In [28]:
ABO3_data = pysic_features_ABO3
ABO3_data[feature_labels] = small_magpie_features[feature_labels]
ABO3_data

,A_Density,B_Density,A_dipole Polarizability,B_dipole Polarizability,A_covalent Radius,B_covalent Radius,A_atomic Radius,B_atomic Radius,A_FirstIonization,B_FirstIonization,...,MagpieData mean SpaceGroupNumber,MagpieData avg_dev SpaceGroupNumber,MagpieData mode SpaceGroupNumber,avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,compound possible,max ionic char,avg ionic char
0,7.520,7.15,192.0,83.00,172,122,1.85,1.40,544.5,652.9,...,86.2,89.04,12.0,1.8,2.4,1.0,1.2,True,0.724240,0.154892
1,2.990,8.86,97.0,55.00,148,111,1.60,1.35,633.1,760.4,...,84.8,87.36,12.0,2.0,2.4,1.6,0.0,True,0.660947,0.136622
2,7.900,7.15,158.0,83.00,169,122,1.80,1.40,593.4,652.9,...,91.8,95.76,12.0,1.8,2.4,1.2,1.4,True,0.714753,0.153484
3,4.470,7.15,162.0,83.00,163,122,1.80,1.40,600.0,652.9,...,91.8,95.76,12.0,1.8,2.4,1.2,0.0,True,0.708321,0.152541
4,6.150,7.87,215.0,62.00,180,116,1.95,1.40,538.1,762.5,...,91.8,95.76,12.0,2.0,2.4,1.4,0.0,True,0.745613,0.151694
5,7.520,8.86,192.0,55.00,172,111,1.85,1.35,544.5,760.4,...,79.2,80.64,12.0,2.0,2.4,1.4,1.2,True,0.724240,0.146339
6,4.470,7.87,162.0,62.00,163,116,1.80,1.40,600.0,762.5,...,91.8,95.76,12.0,2.0,2.4,1.4,0.0,True,0.708321,0.145782
7,6.150,7.15,215.0,83.00,180,122,1.95,1.40,538.1,652.9,...,91.8,95.76,12.0,1.8,2.4,1.2,0.0,True,0.745613,0.158143
8,6.770,7.15,216.0,83.00,176,122,1.85,1.40,527.0,652.9,...,91.8,95.76,12.0,1.8,2.4,1.0,0.6,True,0.736585,0.156756
9,7.010,7.15,208.0,83.00,174,122,1.85,1.40,533.1,652.9,...,91.8,95.76,12.0,1.8,2.4,1.0,0.8,True,0.733532,0.156291


In [30]:
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tqdm import tqdm
train_model = ExtraTreesRegressor(n_jobs = -1, random_state = 42)
y_eg_abo3 = small_data['target']
error1 = []
error2 = []
r2_prm = []
error1_base=[]
error2_base=[]
r2_prl=[]
train_set = []
test_set = []
tmp_df = pd.DataFrame()
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_indices = kf.split(X,y)
base_features = ABO3_data[optimize_feature_labels]
for i,(train_index, test_index) in enumerate(cv_indices):
    train_set.append(train_index)
    test_set.append(test_index)
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    X_train = X_train[optimize_feature_labels]
    X_test = X_test[optimize_feature_labels]
    name = f"fold_{i}"
    train_model.fit(X_train, y_train)
    y_pred =  train_model.predict(X_test)
    mae_model = mean_absolute_error(y_test, y_pred)
    rmse_model = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_model = r2_score(y_test,y_pred)
    error1.append(mae_model)
    error2.append(rmse_model)
    r2_prm.append(r2_model)
    
    y_base = train_model.predict(base_features)
    tmp_df[f'column_{i}'] = y_base
    MAE_BASE = mean_absolute_error(y_base,y_eg_abo3)
    RMSE_BASE = np.sqrt(mean_squared_error(y_base,y_eg_abo3))
    R2_BASE = r2_score(y_base,y_eg_abo3)
    error1_base.append(MAE_BASE)
    error2_base.append(RMSE_BASE)
    r2_prl.append(R2_BASE)
    print(name)
    print(f"Performance on the test set: MAE = %0.4f,RMSE = %0.4f, R2 = %0.4f" % (mae_model, rmse_model, r2_model))
    print(f"Performance on the small set: MAE = %0.4f,RMSE = %0.4f, R2 = %0.4f" % (MAE_BASE, RMSE_BASE, R2_BASE))
mean_error1 = sum(error1) / len(error1)
mean_error2 = sum(error2) / len(error2)
mean_r2 = sum(r2_prm) / len(r2_prm)
print('Performance on the test set using 5-fold: MAE = %0.4f' % mean_error1)
print('Performance on the test set using 5-fold: RMSE = %0.4f' % mean_error2)
print('Performance on the test set using 5-fold: R2 = %0.4f' % mean_r2)

mean_error1_base = sum(error1_base) / len(error1_base)
mean_error2_base = sum(error2_base) / len(error2_base)
mean_r2_base = sum(r2_prl) / len(r2_prl)
print('Performance on the valid set using 5-fold: MAE = %0.4f' % mean_error1_base)
print('Performance on the valid set using 5-fold: RMSE = %0.4f' % mean_error2_base)
print('Performance on the valid set using 5-fold: R2 = %0.4f' % mean_r2_base)

fold_0
Performance on the test set: MAE = 0.3164,RMSE = 0.4393, R2 = 0.8741
Performance on the small set: MAE = 0.4377,RMSE = 0.5532, R2 = 0.6120
fold_1
Performance on the test set: MAE = 0.3262,RMSE = 0.4528, R2 = 0.8653
Performance on the small set: MAE = 0.4178,RMSE = 0.5391, R2 = 0.6361
fold_2
Performance on the test set: MAE = 0.3243,RMSE = 0.4505, R2 = 0.8711
Performance on the small set: MAE = 0.4294,RMSE = 0.5528, R2 = 0.6330
fold_3
Performance on the test set: MAE = 0.3171,RMSE = 0.4402, R2 = 0.8720
Performance on the small set: MAE = 0.4151,RMSE = 0.5417, R2 = 0.6564
fold_4
Performance on the test set: MAE = 0.3152,RMSE = 0.4394, R2 = 0.8731
Performance on the small set: MAE = 0.4292,RMSE = 0.5476, R2 = 0.6464
Performance on the test set using 5-fold: MAE = 0.3199
Performance on the test set using 5-fold: RMSE = 0.4444
Performance on the test set using 5-fold: R2 = 0.8711
Performance on the valid set using 5-fold: MAE = 0.4258
Performance on the valid set using 5-fold: RMSE =

In [31]:
best_train_index = train_set[3]
best_test_index = test_set[3]
best_X_train = X.loc[best_train_index].reset_index(drop=True)
best_X_test = X.loc[best_test_index].reset_index(drop=True)
best_y_train = y.loc[best_train_index].reset_index(drop=True)
best_y_test = y.loc[best_test_index].reset_index(drop=True)

In [32]:
best_X_train = best_X_train[optimize_feature_labels]
best_X_test = best_X_test[optimize_feature_labels]

In [48]:
model_best = ExtraTreesRegressor(n_jobs=-1, random_state=42)
model_best.fit(best_X_train, best_y_train)
y_pred = model_best.predict(best_X_test)
MAE_4 = mean_absolute_error(best_y_test,y_pred)
RMSE_4 = np.sqrt(mean_squared_error(best_y_test, y_pred))
R2_4 = r2_score(best_y_test,y_pred)
print('Performance on the test set: MAE = %0.4f' % MAE_4)
print('Performance on the test set: RMSE = %0.4f' % RMSE_4)
print('Performance on the test set: R2 SCORE = %0.4f' % R2_4)

Performance on the test set: MAE = 0.3171
Performance on the test set: RMSE = 0.4402
Performance on the test set: R2 SCORE = 0.8720


In [38]:
best_f = optimize_feature_labels.tolist()

In [39]:
# import json
# with open("best_feature_m_p.json", "w") as f:
#     json.dump(best_f, f)

In [49]:
# import pickle
# with open('model_m_p.pkl', 'wb') as f:
#     pickle.dump(model_best, f)